In [2]:
import cv2
import numpy as np
import os
import sys
import math

import face_recognition

from typing import Dict, List
from utils import show_image

In [26]:
imgs = {}
for img_name in sorted(os.listdir('validation_folder/images')):
    img_path = os.path.join('validation_folder/images', img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imgs[img_name] = img

In [34]:
K = 5

In [35]:
cluster_results: List[List[str]] = [[]] * K 

In [122]:
def get_distance(face_encoding: np.ndarray, centroids: np.ndarray):
    distances = []
    for centroid in centroids:
        distance = np.linalg.norm(face_encoding - centroid)
        distances.append(distance)
    return distances

In [120]:
def get_centroids(face_encodings: dict, cluster_results):
    centroids = []
    for cluster in cluster_results:
        cluster_images = [face_encodings[i] for i in cluster]
        cluster_images = np.array(cluster_images)
        cluster_images = np.reshape(cluster_images, (cluster_images.shape[0], 128))  
        centroids.append(np.reshape(np.mean(cluster_images, axis=0), (1, 128)))
    centroids = np.array(centroids)
    centroids = np.reshape(centroids, (centroids.shape[0], 128))
    return np.array(centroids)

In [92]:
# Do not use this function
# def get_cluster_results_temp(face_encodings: dict, centroids: dict, K):
#     cluster_results = [[] for i in range(K)]
#     for i in range(face_encodings.shape[0]):
#         face_encoding = face_encodings[i]
#         distances = get_distance(face_encoding, centroids)
#         cluster_results[np.argmin(distances)].append(i)
#     return cluster_results

In [91]:
def get_cluster_results(face_encodings: dict, centroids: np.ndarray, K: int):
    cluster_results = [[] for i in range(K)]
    for i in face_encodings.keys():
        face_encoding = face_encodings[i]
        distances = get_distance(face_encoding, centroids)
        cluster_results[np.argmin(distances)].append(i)
    return cluster_results

In [109]:
count = 0
face_encoding_results = {}
face_encodings = np.array([]).reshape(0, 128)
for img_name in imgs.keys():
    face_encoding = np.array(face_recognition.face_encodings(imgs[img_name]))
    if face_encoding.shape[0] == 0:
        count += 1
        continue
    face_encodings = np.concatenate((face_encodings, face_encoding), axis=0)
    if face_encoding.shape[0] > 1:
        for i in range(face_encoding.shape[0]):
            face_encoding_results[img_name + '_' + str(i)] = np.reshape(face_encoding[i], (1, 128))
    else:
        face_encoding_results[img_name] = np.reshape(face_encoding, (1, 128))

In [126]:
centroids = []
centroids_names = np.random.choice(np.array(list(face_encoding_results.keys())), K, replace=False)
for n in centroids_names:
    centroids.append(face_encoding_results[n])

centroids = np.array(centroids)
centroids = np.reshape(centroids, (centroids.shape[0], 128))
centroids.shape

(5, 128)

In [123]:
cluster_results = get_cluster_results(face_encoding_results, centroids, K)

In [119]:
cluster1 = cluster_results[0]
cluster_images = [face_encoding_results[i] for i in cluster1]
cluster_images = np.array(cluster_images)
cluster_images = np.reshape(cluster_images, (cluster_images.shape[0], 128))  
c = np.reshape(np.mean(cluster_images, axis=0), (1, 128))
c.shape

(1, 128)

In [124]:
while True:
    new_centroids = get_centroids(face_encoding_results, cluster_results)
    new_cluster_results = get_cluster_results(face_encoding_results, new_centroids, K)
    if np.array_equal(cluster_results, new_cluster_results):
        break
    cluster_results = new_cluster_results

c:\Users\prana\.conda\envs\CVIP\lib\site-packages\numpy\core\numeric.py:2440: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a1, a2 = asarray(a1), asarray(a2)
